<a href="https://colab.research.google.com/github/The-cheater/Deep_Learning_Models/blob/main/A_MM_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
import random
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths to zipped datasets
gaf_zip = "/content/drive/MyDrive/dataset/GAF_Images_224x224.zip"
mtf_zip = "/content/drive/MyDrive/dataset/MTF_Images_224x224.zip"

# Extract paths
gaf_extract_path = "/content/GAF_Images_224x224"
mtf_extract_path = "/content/MTF_Images_224x224"
os.makedirs(gaf_extract_path, exist_ok=True)
os.makedirs(mtf_extract_path, exist_ok=True)

with zipfile.ZipFile(gaf_zip, 'r') as zip_ref:
    zip_ref.extractall(gaf_extract_path)

with zipfile.ZipFile(mtf_zip, 'r') as zip_ref:
    zip_ref.extractall(mtf_extract_path)

# ==========================
# Custom Data Generator
# ==========================
class GAFMTFGenerator(Sequence):
    def __init__(self, gaf_dir, mtf_dir, batch_size=32, image_size=(224, 224), shuffle=True):
        self.gaf_dir = gaf_dir
        self.mtf_dir = mtf_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle

        self.samples = []
        self.class_names = set()

        # Navigate into the nested directory structure
        nested_gaf_base = os.path.join(gaf_dir, os.path.basename(gaf_dir))
        nested_mtf_base = os.path.join(mtf_dir, os.path.basename(mtf_dir))

        if not os.path.exists(nested_gaf_base):
             raise FileNotFoundError(f"Nested GAF directory not found: {nested_gaf_base}")
        if not os.path.exists(nested_mtf_base):
             raise FileNotFoundError(f"Nested MTF directory not found: {nested_mtf_base}")


        # Traverse all features (like 'Left Stance Interval') and classes
        for feature in os.listdir(nested_gaf_base):
            gaf_feature_path = os.path.join(nested_gaf_base, feature)
            mtf_feature_path = os.path.join(nested_mtf_base, feature)
            if not os.path.isdir(gaf_feature_path) or not os.path.isdir(mtf_feature_path):
                continue

            for class_name in os.listdir(gaf_feature_path):
                gaf_class_path = os.path.join(gaf_feature_path, class_name)
                mtf_class_path = os.path.join(mtf_feature_path, class_name)
                if not os.path.isdir(gaf_class_path) or not os.path.isdir(mtf_class_path):
                    continue

                self.class_names.add(class_name)

                for filename in os.listdir(gaf_class_path):
                    if filename.endswith('.png') and os.path.exists(os.path.join(mtf_class_path, filename)):
                        self.samples.append((
                            os.path.join(gaf_class_path, filename),
                            os.path.join(mtf_class_path, filename),
                            class_name
                        ))

        self.class_names = sorted(list(self.class_names))
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(self.class_names)

        print(f"✅ Found {len(self.samples)} matched GAF-MTF pairs across {len(self.class_names)} classes.")
        if self.shuffle:
            random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples) // self.batch_size

    def __getitem__(self, idx):
        batch = self.samples[idx * self.batch_size: (idx + 1) * self.batch_size]
        gaf_batch, mtf_batch, labels = [], [], []

        for gaf_path, mtf_path, label in batch:
            gaf_img = img_to_array(load_img(gaf_path, target_size=self.image_size)) / 255.0
            mtf_img = img_to_array(load_img(mtf_path, target_size=self.image_size)) / 255.0
            gaf_batch.append(gaf_img)
            mtf_batch.append(mtf_img)
            labels.append(label)

        gaf_batch = np.array(gaf_batch)
        mtf_batch = np.array(mtf_batch)
        labels = self.label_encoder.transform(labels)
        labels = tf.keras.utils.to_categorical(labels, num_classes=len(self.class_names))

        return {"gaf_input": gaf_batch, "mtf_input": mtf_batch}, labels

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.samples)

# ==========================
# Build Multimodal Model
# ==========================
def build_mmcnn(input_shape=(224, 224, 3), num_classes=4):
    # GAF branch
    gaf_input = Input(shape=input_shape, name='gaf_input')
    gaf_conv = Conv2D(32, (3, 3), activation='relu')(gaf_input)
    gaf_pool = MaxPooling2D((2, 2))(gaf_conv)
    gaf_conv = Conv2D(64, (3, 3), activation='relu')(gaf_pool)
    gaf_pool = MaxPooling2D((2, 2))(gaf_conv)
    gaf_flatten = Flatten()(gaf_pool)

    # MTF branch
    mtf_input = Input(shape=input_shape, name='mtf_input')
    mtf_conv = Conv2D(32, (3, 3), activation='relu')(mtf_input)
    mtf_pool = MaxPooling2D((2, 2))(mtf_conv)
    mtf_conv = Conv2D(64, (3, 3), activation='relu')(mtf_pool)
    mtf_pool = MaxPooling2D((2, 2))(mtf_conv)
    mtf_flatten = Flatten()(mtf_pool)

    # Combined
    combined = concatenate([gaf_flatten, mtf_flatten])
    dense = Dense(128, activation='relu')(combined)
    output = Dense(num_classes, activation='softmax')(dense)

    return Model(inputs=[gaf_input, mtf_input], outputs=output)

# ==========================
# Setup Training
# ==========================
gaf_path = "/content/GAF_Images_224x224"
mtf_path = "/content/MTF_Images_224x224"

train_gen = GAFMTFGenerator(gaf_path, mtf_path, batch_size=16)
model = build_mmcnn(input_shape=(224, 224, 3), num_classes=len(train_gen.class_names))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ==========================
# Train the Model
# ==========================
model.fit(train_gen, epochs=20)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Found 3328 matched GAF-MTF pairs across 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 31s 119ms/step - accuracy: 0.4146 - loss: 3.7334
Epoch 2/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 24s 115ms/step - accuracy: 0.7736 - loss: 0.5746
Epoch 3/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 109ms/step - accuracy: 0.8924 - loss: 0.2732
Epoch 4/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 108ms/step - accuracy: 0.9328 - loss: 0.1609
Epoch 5/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 110ms/step - accuracy: 0.9407 - loss: 0.1226
Epoch 6/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 113ms/step - accuracy: 0.9350 - loss: 0.1372
Epoch 7/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 110ms/step - accuracy: 0.9361 - loss: 0.1388
Epoch 8/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 110ms/step - accuracy: 0.9380 - loss: 0.1421
Epoch 9/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 112ms/step - accuracy: 0.9443 - loss: 0.1090
Epoch 10/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 110ms/step - accuracy: 0.9397 - loss: 0.1220
Epoch 11/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 23s 109ms/step - accuracy: 0.9514 - loss: 0.0965
Epoch 12/20
208/208

In [6]:
from google.colab import files

# Upload both GAF and MTF images
print("📂 Please upload the GAF and MTF images (must be .png)")
uploaded = files.upload()
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Detect which file is GAF and which is MTF based on filename
gaf_img_path = [f for f in uploaded if 'gaf' in f.lower()][0]
mtf_img_path = [f for f in uploaded if 'mtf' in f.lower()][0]

# Load and preprocess
def preprocess_image(path):
    img = load_img(path, target_size=(224, 224))
    arr = img_to_array(img) / 255.0
    return np.expand_dims(arr, axis=0)

gaf_input = preprocess_image(gaf_img_path)
mtf_input = preprocess_image(mtf_img_path)
# Predict using your trained model
prediction = model.predict({"gaf_input": gaf_input, "mtf_input": mtf_input})
predicted_index = np.argmax(prediction)

# Use class labels from your generator
class_labels = train_gen.class_names
predicted_label = class_labels[predicted_index]

print("✅ Predicted Disease Class:", predicted_label)
for label, prob in zip(class_labels, prediction[0]):
    print(f"{label}: {prob:.2%}")


📂 Please upload the GAF and MTF images (must be .png)


Saving test1_gaf.png to test1_gaf (1).png
Saving test2_mtf.png to test2_mtf (1).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
✅ Predicted Disease Class: hunt
als: 13.78%
control: 29.07%
hunt: 32.45%
park: 24.71%
